# Milestone 1

- Currency data

In [1]:
from selenium import webdriver
import pandas as pd

driver = webdriver.Chrome()
driver.get("http://www.bnm.gov.my/?tpl=exchangerates")

currency = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[2]")
date = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[1]/th[2]/b")
buy = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[3]")
sell = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[5]")
middle = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/section/div/div/div/div/div[2]/table[4]/tbody/tr[20]/td[7]")


currency_unit = currency.text
date_day = date.text
buy_rate = buy.text
sell_rate = sell.text
middle_rate = middle.text

dataframe = pd.DataFrame({"currency_unit":currency_unit, "date_day":date_day, "buy_rate":buy_rate, "sell_rate":sell_rate, "middle_rate":middle_rate}, index = [0])
print(dataframe)
# dataframe.to_csv("Day_5.csv")


  currency_unit              date_day buy_rate sell_rate middle_rate
0           USD  31 May 2019 (Friday)   4.1890    4.1920      4.1905


- Stock

In [10]:
# import packages
from selenium import webdriver
from lxml import html
import requests
import pandas as pd

##-----------------------------------------------------------------------------------------------------------
## Crawl for company names
# disable picture
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
browser = webdriver.Chrome(options=chrome_options)
# browser = webdriver.Chrome() # open web page
browser.implicitly_wait(10)  # wait for web page to load

company_names = []  # save company names in a list
# crawl all company names from A-Z
for i in range(65, 91):
    url = 'https://www.thestar.com.my/business/marketwatch/stock-list/?alphabet=' + chr(i)
    browser.get(url)

    name_list = browser.find_elements_by_xpath('//table[@class="market-trans"]//tr[@class="linedlist"]/td/a')
    for name in name_list:
        if name.text != '':
            name_text = name.text.replace("&", "%26")
            company_names.append(name_text)

# crawl all company names in 0-9
url1 = 'https://www.thestar.com.my/business/marketwatch/stock-list/?alphabet=0-9'
browser.get(url1)
name1_list = browser.find_elements_by_xpath('//table[@class="market-trans"]//tr[@class="linedlist"]/td/a')
for name1 in name1_list:
    if name1.text != '':
        name1_text = name1.text.replace("&", "%26")
        company_names.append(name1_text)

# print(company_names)
browser.close()

# save as links for crawling all the information
company_links = []
for n in company_names:
    link = 'https://www.thestar.com.my/business/marketwatch/stocks/?qcounter=' + n
    company_links.append(link)

##-----------------------------------------------------------------------------------------------------------
## Crawl for stock price

header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
        'Referer':'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-2305666475781689&output=html&h=90&slotname=9390552809&adk=3116781417&adf=1662620477&w=970&lmt=1552753083&guci=2.2.0.0.2.2.0.0&format=970x90&url=http%3A%2F%2Fwww.investalks.com%2Fforum%2Fforum.php%3Fmod%3Dforumdisplay%26fid%3D7%26filter%3Dtypeid%26typeid%3D17&flash=0&wgl=1&dt=1552753083912&bpp=42&bdt=108&fdt=46&idt=22&shv=r20190313&cbv=r20190131&saldr=aa&abxe=1&correlator=6683659723222&frm=20&pv=2&ga_vid=691979923.1552569252&ga_sid=1552752744&ga_hid=1232295072&ga_fc=1&iag=0&icsg=12206&dssz=10&mdo=0&mso=0&u_tz=480&u_his=3&u_java=0&u_h=864&u_w=1536&u_ah=824&u_aw=1536&u_cd=24&u_nplug=3&u_nmime=4&adx=34&ady=100&biw=1026&bih=350&scr_x=0&scr_y=0&eid=21060853&oid=3&rx=0&eae=0&fc=656&brdim=426%2C33%2C426%2C33%2C1536%2C0%2C1057%2C735%2C1042%2C350&vis=1&rsz=%7C%7CeE%7C&abl=CS&ppjl=f&pfx=0&fu=16&bc=7&ifi=1&uci=1.afatrway4czq&fsb=1&xpc=kkAgITwzNl&p=http%3A//www.investalks.com&dtd=90'}

class AppCrawler:
    def __init__(self, starting_url, depth):
        self.starting_url = starting_url
        self.depth = depth
        self.apps = []

    def crawl(self):
        self.get_app_from_link(self.starting_url)
        return

    def get_app_from_link(self, link):
        start_page = requests.get(link,headers = header)
        tree = html.fromstring(start_page.text)

        name = tree.xpath('//h1[@class="stock-profile f16"]/text()')[0]
        price = tree.xpath('//td[@id="slcontent_0_ileft_0_lastdonetext"]/text()')[0]
        code = tree.xpath('//li[@class="f14"]/text()')[1]
        date = tree.xpath('//span[@id="slcontent_0_ileft_0_datetxt"]/text()')[0]
        time = tree.xpath('//span[@id="slcontent_0_ileft_0_timetxt"]/text()')[0]
        open_price = tree.xpath('//td[@id="slcontent_0_ileft_0_opentext"]/text()')[0]
        low = tree.xpath('//td[@id="slcontent_0_ileft_0_lowtext"]/text()')[0]
        high = tree.xpath('//td[@id="slcontent_0_ileft_0_hightext"]/text()')[0]
        vol = tree.xpath('//td[@id="slcontent_0_ileft_0_voltext"]/text()')[0]
        buy_vol = tree.xpath('//td[@id="slcontent_0_ileft_0_buyvol"]/text()')[0]
        sell_vol = tree.xpath('//td[@id="slcontent_0_ileft_0_sellvol"]/text()')[0]

        name_list.append(name)
        print(name)
        code_list.append(code[3:])
        # print(code[3:])
        price_list.append(price)
        # print(price)
        date_list.append(date[10:21])
        # print(date[10:21])
        time_list.append(time)
        # print(time)
        open_price_list.append(open_price)
        # print(open_price)
        low_list.append(low)
        # print(low)
        high_list.append(high)
        # print(high)
        vol_list.append(vol)
        # print(vol)
        buy_vol_list.append(buy_vol)
        # print(buy_vol)
        sell_vol_list.append(sell_vol)
        # print(sell_vol)

        return


class App:

    def __init__(self, name, code, price, links):
        self.name = name
        self.code = code
        self.price = price
        self.links = links

    def __str__(self):
        return ("Name: " + self.name.encode('UTF-8') +
                "\r\nCode: " + self.developer.encode('UTF-8') +
                "\r\nPrice: " + self.price.encode('UTF-8') + "\r\n")


# create list for all the variables
name_list = []
code_list = []
price_list = []
date_list = []
time_list = []
open_price_list = []
low_list = []
high_list = []
vol_list = []
buy_vol_list = []
sell_vol_list = []

for l in company_links:
    crawler = AppCrawler(l, 0)
    crawler.crawl()

##-----------------------------------------------------------------------------------------------------------
## Save data as .csv file
na = name_list
co = code_list
da = date_list
ti = time_list
op = open_price_list
lo = low_list
hi = high_list
pr = price_list
vo = vol_list
bv = buy_vol_list
sv = sell_vol_list

dataframe = pd.DataFrame({'name':na,'code':co,'date':da,'time':ti,'open':op,'low':lo,'high':hi,'price':pr,'volume':vo,'buy/volum':bv,'sell/volum':sv})

# save data
print(dataframe)
# dataframe.to_csv("Day_9.csv",index=False,sep=',')


A50CHIN-C26: CW ISHARES FTSE A50 CHINA INDEX ETF (RHB)
A50CHIN-C28: CW ISHARES FTSE A50 CHINA INDEX ETF (RHB)
A50CHIN-C30: CW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
A50CHIN-C32: CW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
A50CHIN-C34: CW ISHARES FTSE A50 CHINA INDEX ETF (RHB)
A50CHIN-C36: CW ISHARES FTSE A50 CHINA INDEX ETF (RHB)
A50CHIN-C38: CW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
A50CHIN-C40: CW ISHARES FTSE A50 CHINA INDEX ETF (RHB)
A50CHIN-H25: PW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
A50CHIN-H27: PW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
A50CHIN-H29: PW ISHARES FTSE A50 CHINA INDEX ETF (MACQ)
ASTRAL ASIA BHD
AIRASIA X BERHAD
AIRASIA X BERHAD - WARRANTS  A  2015/2020
ABF MALAYSIA BOND INDEX FUND
ABLEGROUP BERHAD
ALLIANCE BANK MALAYSIA BERHAD
ABMB-CY: CW ALLIANCE BANK MALAYSIA BERHAD (MIBB)
ABMB-CZ: CW ALLIANCE BANK MALAYSIA BERHAD (KIBB)
ACME HOLDINGS BERHAD
ACOUSTECH BHD
ADVANCECON HOLDINGS BERHAD
ADVENTA BHD
ADVANCED PACKAGING TECHNOLOGY
AE MULTI HOLDINGS BHD
AE MULTI HOLDI

CCK CONSOLIDATED HOLDINGS BERHAD - WARRANTS 2018/2023
CHEMICAL COMPANY OF MALAYSIA
CENSOF HOLDINGS BERHAD
CENSOF HOLDINGS BERHAD - WARRANTS B 14/19
CEPATWAWASAN GROUP BHD
CONCRETE ENGINEERING PRODUCTS
COMPUTER FORMS (M) BHD
CHEETAH HOLDINGS BHD
CHIN HIN GROUP PROPERTY BERHAD
CHIN HIN GROUP PROPERTY BERHAD - WARRANTS 2013/2023
COUNTRY HEIGHTS HOLDINGS BHD
COUNTRY HEIGHTS HOLDINGS BERHAD - WARRANTS B
TRADEPLUS S&P NEW CHINA TRACKER-MYR
TRADEPLUS S&P NEW CHINA TRACKER-USD
CHIN HIN GROUP BERHAD
CHIN TECK PLANTATIONS BHD
CHIN WELL HOLDINGS BHD
CHOO BEE METAL INDUSTRIES BHD
CHUAN HUAT RESOURCES BHD
CENTRAL INDUSTRIAL CORPORATION
C.I. HOLDINGS BHD
CIMB GROUP HOLDINGS BERHAD
CIMB-C41: CW CIMB GROUP HOLDINGS BERHAD (MIBB)
CIMB-C42: CW CIMB GROUP HOLDINGS BERHAD (MIBB)
CIMB-C45: CW CIMB GROUP HOLDINGS BERHAD (MACQ)
CIMB-C46: CW CIMB GROUP HOLDINGS BERHAD (RHB)
CIMB-C47: CW CIMB GROUP HOLDINGS BERHAD (AM)
CIMB-C48: CW CIMB GROUP HOLDINGS BERHAD (MIBB)
CIMB-C49: CW CIMB GROUP HOLDINGS BERHAD (KIBB

FBMKLCI-H6T: PW FTSE BURSA MALAYSIA KLCI INDEX (CIMB)
FBMKLCI-H6U: PW FTSE BURSA MALAYSIA KLCI INDEX (MACQ)
FBMKLCI-H6V: PW FTSE BURSA MALAYSIA KLCI INDEX (MACQ)
FBMKLCI-H6W: PW FTSE BURSA MALAYSIA KLCI INDEX (MIBB)
FBMKLCI-H6X: PW FTSE BURSA MALAYSIA KLCI INDEX (CIMB)
FBMKLCI-H6Y: PW FTSE BURSA MALAYSIA KLCI INDEX (CIMB)
FBMKLCI-H6Z: PW FTSE BURSA MALAYSIA KLCI INDEX (MACQ)
FBMKLCI-H8A: PW FTSE BURSA MALAYSIA KLCI INDEX (MACQ)
FBMKLCI-H8B: PW FTSE BURSA MALAYSIA KLCI INDEX (MIBB)
FBMKLCI-H8C: PW FTSE BURSA MALAYSIA KLCI INDEX (CIMB)
FBMKLCI-H8D: PW FTSE BURSA MALAYSIA KLCI INDEX (CIMB)
FCW HOLDINGS BHD
FGV HOLDINGS BERHAD
FGV-C58: CW FGV HOLDINGS BERHAD (CIMB)
FGV-C62: CW FGV HOLDINGS BERHAD (RHB)
FGV-C63: CW FGV HOLDINGS BERHAD (MACQ)
FGV-C64: CW FGV HOLDINGS BERHAD (RHB)
FGV-C65: CW FGV HOLDINGS BERHAD (KIBB)
FGV-C66: CW FGV HOLDINGS BERHAD (MACQ)
FGV-C67: CW FGV HOLDINGS BERHAD (CIMB)
FGV-C68: CW FGV HOLDINGS BERHAD (MIBB)
FGV-C69: CW FGV HOLDINGS BERHAD (KIBB)
FGV-C70: CW FGV HOLD

HSI-C5J: CW HANG SENG INDEX (MACQ)
HSI-C5K: CW HANG SENG INDEX (MACQ)
HSI-C5L: CW HANG SENG INDEX (MACQ)
HSI-C5M: CW HANG SENG INDEX (MACQ)
HSI-C5N: CW HANG SENG INDEX (MACQ)
HSI-C5O: CW HANG SENG INDEX (MACQ)
HSI-C5P: CW HANG SENG INDEX (MACQ)
HSI-C5Q: CW HANG SENG INDEX (MACQ)
HSI-C5R: CW HANG SENG INDEX (MACQ)
HSI-C5S: CW HANG SENG INDEX (MACQ)
HSI-C5T: CW HANG SENG INDEX (MACQ)
HSI-C5U: CW HANG SENG INDEX (CIMB)
HSI-C5V: CW HANG SENG INDEX (CIMB)
HSI-C5W: CW HANG SENG INDEX (CIMB)
HSI-C5X: CW HANG SENG INDEX (CIMB)
HSI-C5Y: CW HANG SENG INDEX (CIMB)
HSI-C5Z: CW HANG SENG INDEX (CIMB)
HSI-C7A: CW HANG SENG INDEX (MACQ)
HSI-C7B: CW HANG SENG INDEX (MACQ)
HSI-C7C: CW HANG SENG INDEX (MACQ)
HSI-H6H: PW HANG SENG INDEX (MACQ)
HSI-H6I: PW HANG SENG INDEX (MACQ)
HSI-H6J: PW HANG SENG INDEX (MACQ)
HSI-H6K: PW HANG SENG INDEX (MACQ)
HSI-H6L: PW HANG SENG INDEX (MACQ)
HSI-H6M: PW HANG SENG INDEX (MACQ)
HSI-H6N: PW HANG SENG INDEX (MACQ)
HSI-H6O: PW HANG SENG INDEX (MACQ)
HSI-H6P: PW HANG SEN

LHI-CH: CW LEONG HUP INTERNATIONAL BERHAD (MIBB)
LIEN HOE CORPORATION BHD
LII HEN INDUSTRIES BHD
LION FOREST INDUSTRIES BHD
LION INDUSTRIES CORPORATION
LINGKARAN TRANS KOTA HOLDINGS
LKL INTERNATIONAL BERHAD
LONDON BISCUITS BHD
LONDON BISCUITS BERHAD-WARRANTS 2015/2020
LPI CAPITAL BHD
LEADER STEEL HOLDINGS BHD
LTKM BHD
LUSTER INDUSTRIES BHD
LUSTER INDUSTRIES-WARR 12/22
LUSTER INDUSTRIES-WARR 13/23
LUXCHEM CORPORATION BHD
LYC HEALTHCARE BERHAD
LYSAGHT GALVANIZED STEEL BHD
MARINE & GENERAL BERHAD
M3 TECHNOLOGIES(ASIA)BHD
M3 TECHNOLOGIES (ASIA) BERHAD - WARRANTS A 2016/2019
MAA GROUP BERHAD
MACPIE BERHAD
MACPIE BERHAD - WARRANTS 2011 /2021
MAGNA PRIMA BHD
MAGNA PRIMA BERHAD-WARRANTS B 15/20
MAGNI-TECH INDUSTRIES BHD
MAGNUM BERHAD
MAGNUM-C4: CW MAGNUM BERHAD (MIBB)
MAH SING GROUP BHD
MAHSING-C19: CW MAH SING GROUP BERHAD (KIBB)
MAH SING GROUP BERHAD - WARRANTS C
MALAKOFF CORPORATION BERHAD
 MALAKOFC15: CW MALAKOFF CORPORATION BERHAD (CIMB)
MALAKOFC17: CW MALAKOFF CORPORATION BERHAD (MIBB)
M

NEXGRAM HOLDINGS BERHAD
NEXGRAM HOLDINGS BERHAD - WA 12/22
NEXGRAM HOLDINGS BERHAD - WB 13/23
NEXGRAM HOLDINGS BERHAD - WARRANTS C  14/24
NEXTGREEN GLOBAL BERHAD
NEXTGREEN GLOBAL BERHAD - WARRANTS A 2015/2020
NEW HOONG FATT HOLDINGS BHD
NICHE CAPITAL EMAS HLDG BHD
NICHE CAPITAL EMAS HOLDINGS BERHAD - WARRANTS 2019/2024
NI HSIN RESOURCES BHD
NI HSIN RESOURCES BERHAD - WARRANTS 2019/2022
NOTION VTEC BHD
NOTION VTEC BERHAD - WARRANTS C 2018/2023
NOVA WELLNESS GROUP BERHAD
NOVA MSC BHD
NPC RESOURCES BHD
NOVA PHARMA SOLUTIONS BERHAD
NEGRI SEMBILAN OIL PALMS BHD
NTPM HOLDINGS BHD
NWP HOLDINGS BHD
NYLEX (M) BHD
OCB BHD
OCK GROUP BERHAD
OCK GROUP BERHAD - WARRANTS A 2015/2020
OCEANCASH PACIFIC BHD
OCR GROUP BERHAD
OCR GROUP BERHAD - IRREDEEMABLE CONVERTIBLE PREFERENCE SHARE 2016/2021
OCR GROUP BERHAD - WARRANTS 2016/2021
OCR GROUP BERHAD - WARRANTS 2019/2022
ORIENTAL FOOD INDUSTRIES HLDG
ORIENTAL INTEREST BHD
OKA CORPORATION BHD
OLYMPIA INDUSTRIES BHD
OMESTI BHD
OPCOM HOLDINGS BHD
OPENSYS (M) 

SAPNRGC74: CW SAPURA ENERGY BERHAD (KIBB)
SAPNRGC75: CW SAPURA ENERGY BERHAD (KIBB)
SAPNRGC76: CW SAPURA ENERGY BERHAD (MACQ)
SAPNRGC77: CW SAPURA ENERGY BERHAD (MIBB)
SAPURA RESOURCES BHD
SASBADI HOLDINGS BHD
SAUDEE GROUP BERHAD
SAUDEE GROUP BERHAD - WARRANTS 2016/2021
SUNGEI BAGAN RUBBER CO (M) BHD
SBC CORPORATION BHD
SARAWAK CABLE BERHAD
SC ESTATE BUILDER BERHAD
SC ESTATE BUILDER BERHAD - WARRANTS B 2014/2019
SCC HOLDINGS BERHAD
SCGM BHD
SCGM BERHAD - WARRANTS A
SCH GROUP BERHAD
SCH GROUP BERHAD - WARRANTS 2016/2021
SARAWAK CONSOLIDATED IND BHD
SCICOM (MSC) BHD
SCIENTEX BERHAD
SCIENTX-CF: CW SCIENTEX BERHAD (CIMB)
SCANWOLF CORPORATION BHD
SCOMI GROUP BHD
SCOMI GROUP BERHAD - WARRANTS B
SCOMI ENERGY SERVICES BHD
SUPERCOMNET TECHNOLOGIES BHD
SCOPE INDUSTRIES BHD
SCOPE INDUSTRIES BHD - WARR
SELANGOR DREDGING BHD
SEACERA GROUP BERHAD
SEACERA GROUP BERHAD - WARRANTS C
SEAL INCORPORATED BHD
SEALINK INTERNATIONAL BHD
SEREMBAN ENGINEERING BERHAD
SEDANIA INNOVATOR BERHAD
SEE HUP CONSOLIDATED

TM-C41: CW TELEKOM MALAYSIA BERHAD (MACQ)
TM-C42: CW TELEKOM MALAYSIA BERHAD (KIBB)
TM-C43: CW TELEKOM MALAYSIA BERHAD (CIMB)
TM-C44: CW TELEKOM MALAYSIA BERHAD (MIBB)
TMC LIFE SCIENCES BHD
TMC LIFE SCIENCES BERHAD WARRANTS B
TIONG NAM LOGISTICS HOLDINGS
TRANSOCEAN HOLDINGS BHD
TOMEI CONSOLIDATED BHD
TOMYPAK HOLDINGS BHD
TOMYPAK HOLDINGS BERHAD - WARRANTS A 2016/2021
TONG HERR RESOURCES BHD
TOP GLOVE CORPORATION BHD
TOPGLOV-C41: CW TOP GLOVE CORPORATION BHD (CIMB)
TOPGLOV-C42: CW TOP GLOVE CORPORATION BERHAD (MACQ)
TOPGLOV-C43: CW TOP GLOVE CORPORATION BERHAD (KIBB)
TOPGLOV-C44: CW TOP GLOVE CORPORATION BERHAD (MACQ)
TOPGLOV-C45: CW TOP GLOVE CORPORATION BERHAD (MIBB)
TOPGLOV-C46: CW TOP GLOVE CORPORATION BERHAD (MACQ)
TOPGLOV-C47: CW TOP GLOVE CORPORATION BERHAD (RHB)
TOPGLOV-C48: CW TOP GLOVE CORPORATION BERHAD (CIMB)
TOPGLOV-C49: CW TOP GLOVE CORPORATION BERHAD (KIBB)
TOPGLOV-C50: CW TOP GLOVE CORPORATION BERHAD (AM)
TOPGLOV-C51: CW TOP GLOVE CORPORATION BERHAD (MACQ)
TOPGLOV-C52: C